In [1]:
import torch

# 关于randn和rand函数的区别
t=rand返回[0,1)内的随机数。

t=randn从标准正态分布中返回值。

# 当对torch没有设置 requires_grad = True的时候

In [6]:
x1 = torch.randn(3)
y1 = x1 + 2
print(y1)
z1 = x1 * x1 * 2
print(z1)
z2 = z1.mean()
print(z2)

tensor([ 2.8457,  3.7778, -0.7198])
tensor([ 1.4303,  6.3214, 14.7943])
tensor(7.5153)


In [10]:
z2.backward()
x1.grad

RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

# 当没有设置requires_grad = True 的时候，跑反向传播的程序会报错

# 当对torch设置了 requires_grad = True的时候

In [28]:
x1 = torch.randn(3, requires_grad=True)
y1 = x1 + 2
print(y1)
z1 = y1 * y1 *2
print(z1)
z2 = z1.mean()
print(z2)

tensor([1.5305, 0.9524, 2.7910], grad_fn=<AddBackward0>)
tensor([ 4.6847,  1.8142, 15.5790], grad_fn=<MulBackward0>)
tensor(7.3593, grad_fn=<MeanBackward0>)


# 可以发现，当我们设置了requires_grad的时候，tensor数据类型中多了一个grad_fn，当我们相加的时候是AddBackward；
# 相乘的时候是Multiply的Backward；求平均的时候用的是Mean的Backward

In [29]:
z2.backward()
x1.grad

tensor([2.0406, 1.2699, 3.7213])

# 可以发现当设置了requires_grad属性之后，反向传播可以跑通

## 重点注意事项：当最后的z2是一个vector而不是scalar的时候

In [44]:
x1 = torch.randn(3, requires_grad=True)
y1 = x1 + 2
print(y1)
z1 = y1 * y1 *2
print(z1)

tensor([2.5579, 2.5169, 2.8994], grad_fn=<AddBackward0>)
tensor([13.0853, 12.6693, 16.8136], grad_fn=<MulBackward0>)


## 当我们缺少z2 = z1.mean()，输入z1的反向传播的时候，求不出来梯度，因为z1是一个vector

In [41]:
print(z1.dim())
z1.backward()
x1.grad

1


RuntimeError: grad can be implicitly created only for scalar outputs

In [48]:
x1 = torch.randn(3, requires_grad=True)
y1 = x1 + 2
print(y1)
z1 = y1 * y1 *2
print(z1)
v = torch.tensor([1, 1, 1], dtype=torch.float32)
z1.backward(v)
x1.grad

tensor([2.2585, 3.8123, 1.0519], grad_fn=<AddBackward0>)
tensor([10.2014, 29.0668,  2.2129], grad_fn=<MulBackward0>)


tensor([ 9.0339, 15.2491,  4.2075])

# 总结：一般情况下最终生成的函数要是标量，才能使用反向传播确定grad；如果是Vector的话，则需要在backward函数中传入一个梯度
标量的Dimension是0，而Vector的Dimension是1

In [50]:
x = torch.tensor([1.0,2.0],requires_grad=True)
y = (x + 2)**2
z = torch.mean(y)
print(z.dim())
z.backward()    # 这一行等价于对z求x的导数；z = 0.5 * (x+2)^2, 对x求导数之后是x+2，分别带入1和2后，出来的grad就是3和4
x.grad

0


tensor([3., 4.])

In [53]:
x = torch.tensor([1.0,2.0,3.0], requires_grad=True)
y = (x + 2)**2
z = 4*y
z.backward(torch.tensor([2,2,2]))
# 这一行等价于对z求x的导数；z = 4 * (x+2)^2, 对x求导数之后是8(x+2)，分别带入1,2,3后，出来的grad就是24,32,40
# 同时由于z是一个Vector，这样输入代表了梯度，因而最后出来的grad还要分别乘以里面的24，32，40，因而最终结果就是48，64，80
x.grad

tensor([48., 64., 80.])

In [55]:
x = torch.tensor([1.0,2.0,3.0], requires_grad=True)
y = (x + 2)**2
z = 4*y
z.backward(torch.tensor([10,1,11]))
x.grad
# 同理可以看到此时的接过是由24，32，40分别乘以10，1，11出来的结果

tensor([240.,  32., 440.])

# 三种方法避免Pytorch生成梯度函数，影响到之后的梯度计算中。

In [56]:
x = torch.randn(3, requires_grad=True)
x.requires_grad_(False)     # 在变量后面的属性添加一个_，相当于直接该改变该变量x，不需要再次赋值
x

tensor([-0.8449,  0.7260, -0.3099])


In [57]:
x = torch.randn(3, requires_grad=True)
y = x.detach()    # 这个方法会创造一个新的x的复制，只不过此时y没有了requires_grad
y

tensor([-0.1829, -0.2844, -1.1659])

In [59]:
x = torch.randn(3, requires_grad=True)
with torch.no_grad():
    y = x+2
    print(y)
y = x+2
print(y)

tensor([1.5186, 2.9573, 2.8536])
tensor([1.5186, 2.9573, 2.8536], grad_fn=<AddBackward0>)
